In [ ]:
import pandas as pd
import numpy as np
import io
import requests
from scipy.stats import ttest_ind
from statsmodels.stats.proportion import proportions_ztest
import statsmodels.formula.api as smf


#Analyzing the Impact of Charitable Giving Offers

This project aims to analyze the effects of different charitable giving offers on donor behavior using a dataset from a large-scale natural field experiment. The study focuses on understanding how treatment offers, such as matching donation ratios and maximum contribution thresholds, influence donation amounts and participation rates among previous donors.

##Objectives:
Assess Sample Imbalance: Evaluate the balance of pre-treatment covariates between the treatment and control groups to identify any significant differences that may affect the treatment effects.

Estimate Treatment Effects: Calculate and compare the treatment effects on:

Whether a donor gave anything in response to the campaign (binary outcome).
The amount donated by the donor in response to the campaign (continuous outcome).
Investigate Heterogeneous Treatment Effects: Analyze the impact of varying matching offers, maximum donation thresholds, and ask amounts to determine if these factors significantly influence donor behavior.

Include Covariates in Regression Models: Assess the impact of additional covariates on the treatment effects to refine the understanding of factors influencing charitable giving.

##Methodology:

Sample Assessment:

Perform t-tests to compare means of pre-treatment variables (e.g., months since last donation, highest previous contribution) between the treatment and control groups.
Conduct a difference in proportions test for binary variables (e.g., gender, prior donation status) to assess balance.

Estimating Treatment Effects:

Use t-tests to calculate the average treatment effects for both binary and continuous outcomes.
Implement linear regression models using statsmodels to estimate treatment effects while controlling for additional covariates.
Conduct permutation tests to validate the robustness of the treatment effect estimates.

Investigating Heterogeneous Effects:

Perform linear regression analyses to examine the effects of different matching ratios, maximum donation thresholds, and ask amounts on donor behavior.

Incorporating Covariates:

Include relevant covariates in the regression models to identify their effects on the treatment outcomes and adjust the treatment effect estimates accordingly.


In [ ]:
df=pd.read_csv("/content/charitable_data.csv")

df['any_missing'] = df[df.columns[-19:]].max(axis=1)
df = df[df.any_missing==0]


**treatment:** Was the previous donor sent any matching offer?


**treat_ratio2:** Was the matching offer a 2:1 ratio?

**treat_ratio3:** Was the matching offer a 3:1 ratio?

**treat_size25:** Was the maximimation threshold $25k?

**treat_size50:** Was the maximimation threshold $50k?

**treat_size100:** Was the maximimation threshold $100k?

**treat_askd1:** Was the ask amount equal to the highest previous contribution?

**treat_askd2:** Was the ask amount equal to 1.25X the highest previous contribution?

**treat_askd3:** Was the amount equal to 1.5X the highest previous contribution?

**out_amountgive:** how much did the donor give in response to this campaign?

**out_gavedum:** did the member give anything in response to this campaign

**hpa:** highest previous amount

**mrm2:** months since previous donation

**freq:** number of prior donations

**years:** number of years since initial donation

**year5:** donated in 2005

**mrm2:** months since last donation

**female:** donor is female

**couple:** donor is part of couple

**red0:** state was redstate in 2000


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

treatment = df[df['treatment'] == 1]  # Matching offer group
control = df[df['treatment'] == 0]    # No matching offer group

def t_test_and_means(variable):
    treatment_values = treatment[variable].dropna()
    control_values = control[variable].dropna()

    treatment_mean = treatment_values.mean()
    control_mean = control_values.mean()

    t_stat, p_val = ttest_ind(treatment_values, control_values, equal_var=False)

    return treatment_mean, control_mean, t_stat, p_val

variables = ['mrm2', 'hpa', 'freq', 'years']

results = {}

for var in variables:
    treatment_mean, control_mean, t_stat, p_val = t_test_and_means(var)
    results[var] = {
        'treatment_mean': treatment_mean,
        'control_mean': control_mean,
        't-statistic': t_stat,
        'p-value': p_val
    }

for var, stats in results.items():
    print(f"{var}:")
    print(f"  Treatment Mean = {stats['treatment_mean']:.4f}, Control Mean = {stats['control_mean']:.4f}")
    print(f"  t-statistic = {stats['t-statistic']:.4f}, p-value = {stats['p-value']:.4f}")

for var, stats in results.items():
    if stats['p-value'] < 0.05:
        print(f"Imbalance detected for {var}: p-value = {stats['p-value']:.4f}")
    else:
        print(f"No significant imbalance for {var}: p-value = {stats['p-value']:.4f}")


mrm2:
  Treatment Mean = 12.9539, Control Mean = 12.9465
  t-statistic = 0.0622, p-value = 0.9504
hpa:
  Treatment Mean = 59.3578, Control Mean = 58.6564
  t-statistic = 1.0483, p-value = 0.2945
freq:
  Treatment Mean = 8.0599, Control Mean = 8.0636
  t-statistic = -0.0328, p-value = 0.9739
years:
  Treatment Mean = 6.0783, Control Mean = 6.1346
  t-statistic = -1.0240, p-value = 0.3058
No significant imbalance for mrm2: p-value = 0.9504
No significant imbalance for hpa: p-value = 0.2945
No significant imbalance for freq: p-value = 0.9739
No significant imbalance for years: p-value = 0.3058


In [ ]:
binary_vars = ['year5', 'female', 'couple']

results = {}

for var in binary_vars:
    treatment_count = df[df['treatment'] == 1][var].value_counts()
    control_count = df[df['treatment'] == 0][var].value_counts()

    treatment_count = treatment_count.reindex([0, 1], fill_value=0)
    control_count = control_count.reindex([0, 1], fill_value=0)

    count = np.array([treatment_count[1], control_count[1]])
    nobs = np.array([treatment_count.sum(), control_count.sum()])

    z_stat, p_value = proportions_ztest(count, nobs)

    results[var] = {'z-statistic': z_stat, 'p-value': p_value}

for var, res in results.items():
    print(f"{var}: z-statistic = {res['z-statistic']:.4f}, p-value = {res['p-value']:.4f}")

for var, res in results.items():
    if res['p-value'] < 0.05:
        print(f"Evidence of imbalance for {var}: p-value = {res['p-value']:.4f}")
    else:
        print(f"No significant imbalance for {var}: p-value = {res['p-value']:.4f}")

year5: z-statistic = -1.5757, p-value = 0.1151
female: z-statistic = -1.5050, p-value = 0.1323
couple: z-statistic = 0.0274, p-value = 0.9781
No significant imbalance for year5: p-value = 0.1151
No significant imbalance for female: p-value = 0.1323
No significant imbalance for couple: p-value = 0.9781


In [ ]:
outcome_binary = 'out_gavedum'
outcome_amount = 'out_amountgive'

treatment_binary = df[df['treatment'] == 1][outcome_binary]
control_binary = df[df['treatment'] == 0][outcome_binary]

successes_binary = np.array([treatment_binary.sum(), control_binary.sum()])
nobs_binary = np.array([treatment_binary.count(), control_binary.count()])

z_stat_binary, p_value_binary = proportions_ztest(successes_binary, nobs_binary)

avg_treatment_effect_binary = treatment_binary.mean() - control_binary.mean()

treatment_amount = df[df['treatment'] == 1][outcome_amount]
control_amount = df[df['treatment'] == 0][outcome_amount]

t_stat_amount, p_value_amount = ttest_ind(treatment_amount.dropna(), control_amount.dropna())

avg_treatment_effect_amount = treatment_amount.mean() - control_amount.mean()

print(f"Binary Outcome (Did the member give anything?):")
print(f"Average Treatment Effect = {avg_treatment_effect_binary:.4f}")
print(f"Z-statistic = {z_stat_binary:.4f}, p-value = {p_value_binary:.4f}")

print(f"\nContinuous Outcome (How much did the donor give?):")
print(f"Average Treatment Effect = {avg_treatment_effect_amount:.4f}")
print(f"T-statistic = {t_stat_amount:.4f}, p-value = {p_value_amount:.4f}")

if p_value_binary < 0.05:
    print("Statistically significant treatment effect for binary outcome.")
else:
    print("No statistically significant treatment effect for binary outcome.")

if p_value_amount < 0.05:
    print("Statistically significant treatment effect for continuous outcome.")
else:
    print("No statistically significant treatment effect for continuous outcome.")

Binary Outcome (Did the member give anything?):
Average Treatment Effect = 0.0049
Z-statistic = 3.4517, p-value = 0.0006

Continuous Outcome (How much did the donor give?):
Average Treatment Effect = 0.1829
T-statistic = 2.1168, p-value = 0.0343
Statistically significant treatment effect for binary outcome.
Statistically significant treatment effect for continuous outcome.


In [ ]:
#Example code
"""
mod = smf.ols(formula="y ~d", data=df)
res = mod.fit(cov_type='HC3')
print(res.summary())
"""

'\nmod = smf.ols(formula="y ~d", data=df)\nres = mod.fit(cov_type=\'HC3\')\nprint(res.summary())\n'

In [ ]:
model_binary = smf.ols(formula="out_gavedum ~ treatment", data=df)
results_binary = model_binary.fit(cov_type='HC3')

coef_binary = results_binary.params['treatment']
conf_int_binary = results_binary.conf_int().loc['treatment']

model_amount = smf.ols(formula="out_amountgive ~ treatment", data=df)
results_amount = model_amount.fit(cov_type='HC3')

coef_amount = results_amount.params['treatment']
conf_int_amount = results_amount.conf_int().loc['treatment']

print("Binary Outcome: Treatment Effect Coefficient = {:.4f}".format(coef_binary))
print("95% Confidence Interval for Binary Outcome: ({:.4f}, {:.4f})".format(conf_int_binary[0], conf_int_binary[1]))

print("\nContinuous Outcome: Treatment Effect Coefficient = {:.4f}".format(coef_amount))
print("95% Confidence Interval for Continuous Outcome: ({:.4f}, {:.4f})".format(conf_int_amount[0], conf_int_amount[1]))

if conf_int_binary[0] <= 0 <= conf_int_binary[1]:
    print("The confidence interval for the binary outcome crosses 0.")
else:
    print("The confidence interval for the binary outcome does not cross 0.")

if conf_int_amount[0] <= 0 <= conf_int_amount[1]:
    print("The confidence interval for the continuous outcome crosses 0.")
else:
    print("The confidence interval for the continuous outcome does not cross 0.")


Binary Outcome: Treatment Effect Coefficient = 0.0049
95% Confidence Interval for Binary Outcome: (0.0022, 0.0075)

Continuous Outcome: Treatment Effect Coefficient = 0.1829
95% Confidence Interval for Continuous Outcome: (0.0198, 0.3460)
The confidence interval for the binary outcome does not cross 0.
The confidence interval for the continuous outcome does not cross 0.


In [ ]:
n_iterations = 1000
control_ratio = 0.333
treatment_ratio = 0.667

ate_binary = []
ate_amount = []

original_effect_binary = df['out_gavedum'].mean() - df[df['treatment'] == 0]['out_gavedum'].mean()
original_effect_amount = df['out_amountgive'].mean() - df[df['treatment'] == 0]['out_amountgive'].mean()

for _ in range(n_iterations):
    shuffled_treatment = np.random.choice([0, 1], size=len(df), p=[control_ratio, treatment_ratio])
    df['shuffled_treatment'] = shuffled_treatment

    ate_binary.append(df['out_gavedum'].mean() - df[df['shuffled_treatment'] == 0]['out_gavedum'].mean())
    ate_amount.append(df['out_amountgive'].mean() - df[df['shuffled_treatment'] == 0]['out_amountgive'].mean())

ate_binary = np.array(ate_binary)
ate_amount = np.array(ate_amount)

p_value_binary = np.mean(np.abs(ate_binary) >= np.abs(original_effect_binary))
p_value_amount = np.mean(np.abs(ate_amount) >= np.abs(original_effect_amount))

print(f"Estimated p-value for binary outcome: {p_value_binary:.4f}")
print(f"Estimated p-value for continuous outcome: {p_value_amount:.4f}")

Estimated p-value for binary outcome: 0.0010
Estimated p-value for continuous outcome: 0.0300


In [ ]:
mod_offer = smf.ols(formula="out_gavedum ~ treat_ratio2 + treat_ratio3", data=df)
res_offer = mod_offer.fit(cov_type='HC3')

print(res_offer.summary())

mod_threshold = smf.ols(formula="out_gavedum ~ treat_size25 + treat_size50 + treat_size100 + treat_sizeno", data=df)
res_threshold = mod_threshold.fit(cov_type='HC3')

print(res_threshold.summary())

mod_ask = smf.ols(formula="out_gavedum ~ treat_askd1 + treat_askd2 + treat_askd3", data=df)
res_ask = mod_ask.fit(cov_type='HC3')

print(res_ask.summary())

                            OLS Regression Results                            
Dep. Variable:            out_gavedum   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     5.295
Date:                Mon, 30 Sep 2024   Prob (F-statistic):            0.00502
Time:                        21:45:51   Log-Likelihood:                 24504.
No. Observations:               46513   AIC:                        -4.900e+04
Df Residuals:                   46510   BIC:                        -4.897e+04
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.0189      0.001     22.325   

In [ ]:
mod_covariates = smf.ols(formula="out_gavedum ~ treat_ratio2 + treat_ratio3 + treat_size25 + treat_size50 + treat_size100 + treat_sizeno + treat_askd1 + treat_askd2 + treat_askd3 + year5 + female + couple + mrm2 + hpa + freq + years", data=df)
res_covariates = mod_covariates.fit(cov_type='HC3')

print(res_covariates.summary())

                            OLS Regression Results                            
Dep. Variable:            out_gavedum   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     25.23
Date:                Mon, 30 Sep 2024   Prob (F-statistic):           4.62e-71
Time:                        21:45:51   Log-Likelihood:                 24954.
No. Observations:               46513   AIC:                        -4.988e+04
Df Residuals:                   46497   BIC:                        -4.974e+04
Df Model:                          15                                         
Covariance Type:                  HC3                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.0212      0.002     13.675

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 16, but rank is 15
  warnings.warn('covariance of constraints does not have full '


The results show that after including the covariates, the treatment effects (matching ratios, size, and ask amount) remain statistically insignificant. However, the individual donor characteristics, especially the frequency of prior donations and the time since the last donation, have a significant effect on the likelihood of donating.

In [ ]:
correlation_columns = ['out_gavedum', 'year5', 'female', 'couple', 'mrm2', 'hpa', 'freq', 'years']
correlation_df = df[correlation_columns]

correlation_matrix = correlation_df.corr()

print(correlation_matrix)

             out_gavedum     year5    female    couple      mrm2       hpa  \
out_gavedum     1.000000  0.015355  0.005214  0.000289 -0.070025  0.011068   
year5           0.015355  1.000000 -0.043266  0.064989  0.054198  0.205799   
female          0.005214 -0.043266  1.000000 -0.198173 -0.013517 -0.014613   
couple          0.000289  0.064989 -0.198173  1.000000 -0.028001  0.038112   
mrm2           -0.070025  0.054198 -0.013517 -0.028001  1.000000  0.007017   
hpa             0.011068  0.205799 -0.014613  0.038112  0.007017  1.000000   
freq            0.112431  0.486505  0.015323  0.060945 -0.116325  0.201573   
years           0.021111  0.758127  0.003122  0.078916  0.083117  0.230623   

                 freq     years  
out_gavedum  0.112431  0.021111  
year5        0.486505  0.758127  
female       0.015323  0.003122  
couple       0.060945  0.078916  
mrm2        -0.116325  0.083117  
hpa          0.201573  0.230623  
freq         1.000000  0.643789  
years        0.643789  1.

In [ ]:
red_state_data = df[df['red0'] == 1]
blue_state_data = df[df['red0'] == 0]

mod_red = smf.ols(formula="out_gavedum ~ treatment", data=red_state_data)
res_red = mod_red.fit(cov_type='HC3')
print("Red State Results:")
print(res_red.summary())

mod_blue = smf.ols(formula="out_gavedum ~ treatment", data=blue_state_data)
res_blue = mod_blue.fit(cov_type='HC3')
print("Blue State Results:")
print(res_blue.summary())

red_ci = res_red.conf_int()
blue_ci = res_blue.conf_int()
print("Red State Confidence Interval:")
print(red_ci)
print("Blue State Confidence Interval:")
print(blue_ci)


Red State Results:
                            OLS Regression Results                            
Dep. Variable:            out_gavedum   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     23.47
Date:                Mon, 30 Sep 2024   Prob (F-statistic):           1.28e-06
Time:                        21:45:51   Log-Likelihood:                 9698.4
No. Observations:               18611   AIC:                        -1.939e+04
Df Residuals:                   18609   BIC:                        -1.938e+04
Df Model:                           1                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0144      0.002 

In [ ]:
mod_interaction = smf.ols(formula="out_gavedum ~ treatment * red0", data=df)
res_interaction = mod_interaction.fit(cov_type='HC3')

print(res_interaction.summary())

interaction_ci = res_interaction.conf_int()
print("Interaction Term Confidence Interval:")
print(interaction_ci)


                            OLS Regression Results                            
Dep. Variable:            out_gavedum   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     8.159
Date:                Mon, 30 Sep 2024   Prob (F-statistic):           1.99e-05
Time:                        21:45:51   Log-Likelihood:                 24509.
No. Observations:               46513   AIC:                        -4.901e+04
Df Residuals:                   46509   BIC:                        -4.897e+04
Df Model:                           3                                         
Covariance Type:                  HC3                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.0197      0.001     13.

Interaction Term Significance: The interaction term (treatment × red state) is statistically significant, with a p-value of 0.002. This indicates that the effect of the treatment is different in red states compared to blue states.
Magnitude of Effect in Red States: The treatment effect increases the likelihood of giving in red states by 0.0085 (or 0.85%). The 95% confidence interval for this effect (0.0031 to 0.0138) does not include zero, confirming that this result is statistically significant.
Negligible Effect in Blue States: In blue states, the treatment effect is almost zero (0.0015) and not statistically significant, as indicated by the p-value of 0.406.
Confidence Interval of Interaction: The confidence interval of the interaction term (0.0031 to 0.0138) supports that the treatment effect is significantly larger in red states than in blue states.